# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import timedelta, datetime

In [2]:
a = pd.DataFrame({'A':[1,2,11],'B':[8,2,-8]})

In [8]:
(a>0).any(axis=1)

0    True
1    True
2    True
dtype: bool

# Read Sample

In [2]:
# flights_sample = pd.read_csv(r"../data/1_sampled data/3_percent.csv")
flights_sample = pd.read_csv(r"flights_test.csv")
flights_sample.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance
0,0,2020-01-01,WN,WN,WN,5888,WN,N951WN,5888,13891,...,"Ontario, CA",14771,SFO,"San Francisco, CA",1810,1945,N,95,1,363
1,1,2020-01-01,WN,WN,WN,6276,WN,N467WN,6276,13891,...,"Ontario, CA",14771,SFO,"San Francisco, CA",1150,1320,N,90,1,363
2,2,2020-01-01,WN,WN,WN,4598,WN,N7885A,4598,13891,...,"Ontario, CA",14831,SJC,"San Jose, CA",2020,2130,N,70,1,333
3,3,2020-01-01,WN,WN,WN,4761,WN,N551WN,4761,13891,...,"Ontario, CA",14831,SJC,"San Jose, CA",1340,1455,N,75,1,333
4,4,2020-01-01,WN,WN,WN,5162,WN,N968WN,5162,13891,...,"Ontario, CA",14831,SJC,"San Jose, CA",915,1035,N,80,1,333


In [3]:
flights_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660556 entries, 0 to 660555
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Unnamed: 0          660556 non-null  int64 
 1   fl_date             660556 non-null  object
 2   mkt_unique_carrier  660556 non-null  object
 3   branded_code_share  660556 non-null  object
 4   mkt_carrier         660556 non-null  object
 5   mkt_carrier_fl_num  660556 non-null  int64 
 6   op_unique_carrier   660556 non-null  object
 7   tail_num            659057 non-null  object
 8   op_carrier_fl_num   660556 non-null  int64 
 9   origin_airport_id   660556 non-null  int64 
 10  origin              660556 non-null  object
 11  origin_city_name    660556 non-null  object
 12  dest_airport_id     660556 non-null  int64 
 13  dest                660556 non-null  object
 14  dest_city_name      660556 non-null  object
 15  crs_dep_time        660556 non-null  int64 
 16  cr

In [4]:
flights_sample.loc[:,["fl_date","crs_arr_time","crs_dep_time"]] = flights_sample[["fl_date","crs_arr_time","crs_dep_time"]].astype('string')

arr_time = pd.to_datetime(flights_sample["fl_date"]+" "+ flights_sample["crs_arr_time"].str.zfill(4).replace({'2400':'0000'}))
dep_time = pd.to_datetime(flights_sample["fl_date"]+" "+ flights_sample["crs_dep_time"].str.zfill(4).replace({'2400':'0000'}))

flights_sample["arr_date_time"] = arr_time
flights_sample["dep_date_time"] = dep_time

flights_sample = flights_sample.astype({'arr_date_time':'datetime64[ns]','dep_date_time':'datetime64[ns]'})

In [5]:
flights_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660556 entries, 0 to 660555
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Unnamed: 0          660556 non-null  int64         
 1   fl_date             660556 non-null  string        
 2   mkt_unique_carrier  660556 non-null  object        
 3   branded_code_share  660556 non-null  object        
 4   mkt_carrier         660556 non-null  object        
 5   mkt_carrier_fl_num  660556 non-null  int64         
 6   op_unique_carrier   660556 non-null  object        
 7   tail_num            659057 non-null  object        
 8   op_carrier_fl_num   660556 non-null  int64         
 9   origin_airport_id   660556 non-null  int64         
 10  origin              660556 non-null  object        
 11  origin_city_name    660556 non-null  object        
 12  dest_airport_id     660556 non-null  int64         
 13  dest                660556 no

In [6]:
flights_sample.shape

(660556, 23)

In [7]:
flights_sample = flights_sample[(flights_sample.arr_date_time>=datetime(2020,1,1))& (flights_sample.arr_date_time<=datetime(2020,1,7))]

In [8]:
flights_sample.shape

(129736, 23)

# Feature Engineering

In [9]:
flights_fe = flights_sample.copy()

## Airport Type

In [10]:
origin_airport = flights_fe["origin_airport_id"].value_counts()
dest_airport = flights_fe["dest_airport_id"].value_counts()

airports = pd.concat([origin_airport,dest_airport],axis=1)
airports = airports.sort_values(by=["origin_airport_id"],ascending=False)

df1, df2, df3, df4, df5 = np.array_split(airports, 5)

df1["Type"] = "Very Large"
df2["Type"] = "Large"
df3['Type'] = "Medium"
df4["Type"] = "Small"
df5["Type"] = "Very Small"

airport_type = pd.concat([df1,df2,df3,df4,df5]).drop(columns=["origin_airport_id","dest_airport_id"]).to_dict()['Type']

flights_fe["origin_airport_type"] = flights_fe.origin_airport_id.map(lambda x: airport_type[x] )
flights_fe["dest_airport_type"] = flights_fe.dest_airport_id.map(lambda x: airport_type[x] )

## number of inbound flights

In [11]:
def get_num_inbound_flights_carrier(data,minutes=-60,percentages_to_print=[],num_row_processed=[0]):

    if num_row_processed[0] in percentages_to_print:
        print(f"{datetime.now().time()} : {100*num_row_processed[0]/total_num}% rows processed")
    
    num_row_processed[0] = num_row_processed[0]+1
    carrier = data.mkt_unique_carrier
    airport = data.origin_airport_id
    date = data.dep_date_time
    fl_num = data.op_carrier_fl_num
    date_diff = date + timedelta(minutes=minutes)

    mask = ( 
    (flights_fe.mkt_unique_carrier==carrier) & (flights_fe.dest_airport_id==airport) & 
    (flights_fe.arr_date_time >= date_diff) & (flights_fe.arr_date_time <= date) & (flights_fe.op_carrier_fl_num != fl_num)
       )
    return flights_fe[mask].shape[0]


In [12]:
num_row_processed = [0]
total_num = flights_fe.shape[0]
percentages = np.round(np.linspace(0,total_num,11))
flights_fe["inbound_flights"] = flights_fe.apply(get_num_inbound_flights_carrier,axis=1,percentages_to_print=percentages,num_row_processed = num_row_processed)

07:15:29.917066 : 0.0% rows processed
07:17:44.885308 : 10.000308318431276% rows processed
07:19:51.381030 : 19.999845840784364% rows processed
07:21:53.313018 : 30.000154159215636% rows processed
07:23:55.507104 : 39.99969168156873% rows processed
07:25:58.039589 : 50.0% rows processed
07:28:00.448541 : 60.00030831843127% rows processed
07:30:02.570346 : 69.99984584078436% rows processed
07:32:04.660051 : 80.00015415921564% rows processed
07:34:04.634270 : 89.99969168156872% rows processed


In [13]:
flights_fe.shape

(129736, 26)

## Number of flights scheduled to take off from the airport -15 mins from dep_time

In [14]:
def get_num_dep_flights(data,minutes=-30,percentages_to_print=[],num_row_processed=[0]):
    
    if num_row_processed[0] in percentages_to_print:
        print(f"{datetime.now().time()} : {100*num_row_processed[0]/total_num}% rows processed")
    num_row_processed[0] = num_row_processed[0]+1
    
    airport = data.origin
    date = data.dep_date_time
    date_diff = date + timedelta(minutes=minutes)

    mask = ( (flights_fe.origin ==airport) & 
    (flights_fe.dep_date_time >= date_diff) & (flights_fe.dep_date_time <= date)
       )
    return flights_fe[mask].shape[0]-1


In [15]:
num_row_processed = [0]
total_num = flights_fe.shape[0]
percentages = np.round(np.linspace(0,total_num,11))
flights_fe.loc[:,"dep_flights_count"] = flights_fe.apply(get_num_dep_flights,axis=1,percentages_to_print=percentages,num_row_processed = num_row_processed)

07:36:05.217753 : 0.0% rows processed
07:37:59.891241 : 10.000308318431276% rows processed
07:39:56.946300 : 19.999845840784364% rows processed
07:41:59.406328 : 30.000154159215636% rows processed
07:44:04.988670 : 39.99969168156873% rows processed
07:46:09.902205 : 50.0% rows processed
07:48:11.649657 : 60.00030831843127% rows processed
07:50:12.581256 : 69.99984584078436% rows processed
07:52:18.309103 : 80.00015415921564% rows processed
07:54:38.017497 : 89.99969168156872% rows processed


In [16]:
flights_fe.shape

(129736, 27)

In [17]:
2+3

5

## Number of flights scheduled to arrive to airport +/-20 mins of scheduled arr_time

In [ ]:
pd.Data

In [18]:
def get_num_arr_flights(data,minutes=30,percentages_to_print=[],num_row_processed=[0]):
    
    if num_row_processed[0] in percentages_to_print:
        print(f"{datetime.now().time()} : {100*num_row_processed[0]/total_num}% rows processed")
    num_row_processed[0] = num_row_processed[0]+1
    
    airport = data.dest
    date = data.arr_date_time + timedelta(minutes=minutes)
    date_diff = date - timedelta(minutes=minutes)

    mask = ( (flights_fe.dest ==airport) & 
    (flights_fe.arr_date_time >= date_diff) & (flights_fe.arr_date_time <= date)
       )
    return flights_fe[mask].shape[0]-1

In [19]:
num_row_processed = [0]
total_num = flights_fe.shape[0]
percentages = np.round(np.linspace(0,total_num,11))
flights_fe.loc[:,"arr_flights_count"] = flights_fe.apply(get_num_arr_flights,axis=1,percentages_to_print=percentages,num_row_processed = num_row_processed)

07:56:45.306798 : 0.0% rows processed
07:58:51.939740 : 10.000308318431276% rows processed
08:01:03.336819 : 19.999845840784364% rows processed
08:03:06.364041 : 30.000154159215636% rows processed
08:05:09.791972 : 39.99969168156873% rows processed
08:07:12.203659 : 50.0% rows processed
08:09:15.666966 : 60.00030831843127% rows processed
08:11:17.917104 : 69.99984584078436% rows processed
08:13:19.157909 : 80.00015415921564% rows processed
08:15:23.291126 : 89.99969168156872% rows processed


In [20]:
flights_fe.shape

(129736, 28)

## State names

In [21]:
flights_fe["origin_state"] = flights_fe.origin_city_name.map(lambda x: x.split(", ")[1])
flights_fe["dest_state"] = flights_fe.dest_city_name.map(lambda x: x.split(", ")[1])

In [22]:
flights_fe.origin_state.unique()

array(['CA', 'VA', 'FL', 'OR', 'PA', 'AZ', 'RI', 'ME', 'NC', 'NV', 'NY',
       'TX', 'KY', 'WA', 'PR', 'UT', 'MO', 'OK', 'MN', 'IL', 'SC', 'LA',
       'OH', 'KS', 'WI', 'IN', 'IA', 'MS', 'AR', 'CO', 'MI', 'SD', 'GA',
       'ND', 'NE', 'MT', 'AL', 'TN', 'DC', 'NM', 'MA', 'VT', 'CT', 'HI',
       'NH', 'ID', 'NJ', 'MD', 'WY', 'VI', 'WV', 'AK', 'TT'], dtype=object)

In [23]:
flights_fe.shape

(129736, 30)

## Weather

https://smoosavi.org/datasets/lstw

In [24]:
# # Drop Alska, hawaii, Puerto Rico, U.S. Virgin Islands and American Samoa = AK,HI, DC, GU, MH, MH, MP, PR, VI
# flights_fe = flights_fe[(~flights_fe.origin_state.isin(["AK","HI","DC","PR","VI","TT"])) &
#             (~flights_fe.dest_state.isin(["AK","HI","DC","PR","VI","TT"]))]

In [27]:
weather_kaggle = pd.read_csv(r"../data/external/WeatherEvents_Jan2016-Dec2020.csv")

In [28]:
weather_kaggle = weather_kaggle.astype({'StartTime(UTC)':'datetime64[ns]',
                                       'EndTime(UTC)':'datetime64[ns]'})

In [29]:
weather_kaggle = weather_kaggle[(weather_kaggle["StartTime(UTC)"]>datetime(year=2019,month=1,day=1))&
                                (weather_kaggle["EndTime(UTC)"]<datetime(year=2020,month=2,day=1))]

In [30]:
weather_kaggle.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
1274,W-1275,Snow,Light,2019-01-02 01:54:00,2019-01-02 02:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1275,W-1276,Cold,Severe,2019-01-03 10:54:00,2019-01-03 11:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1276,W-1277,Fog,Moderate,2019-01-03 12:56:00,2019-01-03 13:18:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1277,W-1278,Cold,Severe,2019-01-03 13:18:00,2019-01-03 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1278,W-1279,Snow,Light,2019-01-03 19:54:00,2019-01-03 20:18:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [31]:
flights_fe.origin = flights_fe.origin.replace({'FCA':'GPI','SCE':'UNV','BKG':'BBG','AZA':'IWA','USA':'JQF',
                      'MQT':'SAW','YUM':'NYL','HHH':'HXD'})
flights_fe.dest = flights_fe.dest.replace({'FCA':'GPI','SCE':'UNV','BKG':'BBG','AZA':'IWA','USA':'JQF',
                      'MQT':'SAW','YUM':'NYL','HHH':'HXD'})

In [32]:
def get_weather(data,percentages_to_print=[],num_row_processed=[0]):
    if num_row_processed[0] in percentages_to_print:
        print(f"{datetime.now().time()} : {100*num_row_processed[0]/total_num}% rows processed")
    num_row_processed[0] = num_row_processed[0]+1
    
    arr_time = data.arr_date_time
    dep_time = data.dep_date_time
    origin_id = data.origin
    dest_id = data.dest
    
    weather_details = pd.Series(dtype='string')
    
    try:
        weather_details["arr_type"] ,weather_details["arr_severity"] = weather_kaggle[(weather_kaggle["StartTime(UTC)"]<arr_time)&
                  (weather_kaggle["AirportCode"]=="K"+dest_id)].sort_values(by='StartTime(UTC)',ascending=False).iloc[0][['Type','Severity']]
    except IndexError as e:
        weather_details["arr_type"] ,weather_details["arr_severity"] = np.nan,np.nan
        
    try:
         weather_details["dep_type"] ,weather_details["dep_severity"] = weather_kaggle[(weather_kaggle["StartTime(UTC)"]<dep_time)&
                  (weather_kaggle["AirportCode"]=="K"+origin_id)].sort_values(by='StartTime(UTC)',ascending=False).iloc[0][['Type','Severity']]
    except IndexError as e:
        weather_details["dep_type"] ,weather_details["dep_severity"] = np.nan, np.nan
    
    print(num_row_processed)
    return weather_details

In [33]:
# num_row_processed = [0]
# total_num = flights_fe.shape[0]
# percentages = np.round(np.linspace(0,total_num,11))
# weather_details = flights_fe.apply(get_weather,axis=1,percentages_to_print=percentages,num_row_processed = num_row_processed)

In [34]:
# weather_details.head()

NameError: name 'weather_details' is not defined

In [ ]:
# flights_fe = pd.merge(left=flights_fe,right=weather_details,left_index=True,right_index=True)

## Traffic

## City Population

In [41]:
data = pd.read_csv(r"../data/external/census_data.csv")

In [42]:
population = data.groupby("STNAME").sum()
population.head()

,POPESTIMATE2019
STNAME,
Alabama,9806370
Alaska,1463090
Arizona,14557434
Arkansas,6035608
California,79024446


In [45]:
codes = pd.read_csv(r"../data/external/state codes.csv")

In [46]:
population = pd.merge(left=population,right=codes,right_on='Name',left_index=True)

In [47]:
population = population.set_index('State or Region Code')

In [48]:
population.loc["DC","POPESTIMATE2019"] = 705749
population.loc["VI","POPESTIMATE2019"] = 104578
population.loc["PR","POPESTIMATE2019"] = 3654474
population.loc["TT","POPESTIMATE2019"] = 1394969

In [49]:
flights_fe["origin_population"] = flights_fe.origin_state.map(lambda x: population.loc[x]["POPESTIMATE2019"])

In [50]:
flights_fe["dest_population"] = flights_fe.origin_state.map(lambda x: population.loc[x]["POPESTIMATE2019"])

## Enplanements

In [51]:
enplanemntes = pd.read_excel("../data/external/preliminary-cy20-commercial-service-enplanements.xlsx")

In [52]:
enplanemntes = enplanemntes[["Locid","CY 19 Enplanements"]]
enplanemntes.head()

,Locid,CY 19 Enplanements
0,ATL,53505795.0
1,DFW,35778573.0
2,DEN,33592945.0
3,ORD,40871223.0
4,LAX,42939104.0


In [53]:
ser = flights_fe["origin"] 

In [54]:
flights_fe = pd.merge(left=flights_fe,right=enplanemntes,how="left",left_on="origin",right_on="Locid")

In [55]:
flights_fe.shape

(129736, 36)

# Airport Location details

## Airport details (number of gates, waiting area etc.,)

In [58]:
airport_details = pd.read_csv(r"../data/external/faa_airport_details.csv")
airport_details = airport_details.set_index("airport_code")
airport_details.area = airport_details.area.replace({"":np.nan})
airport_details.head()

,site_id,npias_no,service_level,hub_type,airport_status,lat,lon,city,county,elevation,...,beacon_color,air_carrier_annual,total_operations_annual,operations_annual_ending,num_runways,avg_runway_length,avg_runway_width,Rank,CY 19 Enplanements,Locid
airport_code,,,,,,,,,,,,,,,,,,,,,
ATL,03640.*A,13-0008,Primary,Large,Operational,"33° 38' 12.1186"" N","84° 25' 40.3104"" W","ATLANTA, GA",FULTON,1026.2,...,Clear and Green,474822,525305,01/31/2021,5,9877.800000,150.000000,1.0,53505795.0,ATL
DTW,09749.*A,26-0026,Primary,Large,Operational,"42° 12' 44.8"" N","83° 21' 12.2"" W","DETROIT, MI",WAYNE,645.2,...,Clear and Green,315995,393681,12/31/2018,6,9618.833333,158.333333,18.0,18143040.0,DTW
ATW,27013.1*A,55-0002,Primary,Non-Hub,Operational,"44° 15' 29.1"" N","88° 31' 8.7"" W","APPLETON, WI",OUTAGAMIE,918.3,...,Clear and Green,6490,37809,12/31/2019,2,7251.500000,150.000000,144.0,386737.0,ATW
CSG,03728.*A,13-0035,Primary,Non-Hub,Operational,"32° 30' 58.8"" N","84° 56' 19.9"" W","COLUMBUS, GA",MUSCOGEE,397.4,...,Clear and Green,348,36760,12/31/2019,2,5497.000000,112.500000,266.0,52351.0,CSG
BTR,07425.*A,22-0006,Primary,Non-Hub,Operational,"30° 31' 58.5"" N","91° 8' 59.6"" W","BATON ROUGE, LA",EAST BATON ROUGE,69.7,...,Clear and Green,11908,55331,08/31/2018,3,6101.333333,125.000000,150.0,399591.0,BTR


In [59]:
airport_details = airport_details.drop(columns=['site_id', 'npias_no','airport_status','lat', 'lon', 'city', 'county',
                         'variation',
       'last_inspection_date','operations_annual_ending','elevation','air_carrier_annual','Locid','CY 19 Enplanements'])
airport_details.head()

,service_level,hub_type,ownership_status,facility_use,area,fss_on_airport,beacon_color,total_operations_annual,num_runways,avg_runway_length,avg_runway_width,Rank
airport_code,,,,,,,,,,,,
ATL,Primary,Large,PUBLIC,PUBLIC,4700.0,NO,Clear and Green,525305,5,9877.800000,150.000000,1.0
DTW,Primary,Large,PUBLIC,PUBLIC,4850.0,NO,Clear and Green,393681,6,9618.833333,158.333333,18.0
ATW,Primary,Non-Hub,PUBLIC,PUBLIC,1638.0,NO,Clear and Green,37809,2,7251.500000,150.000000,144.0
CSG,Primary,Non-Hub,PUBLIC,PUBLIC,680.0,NO,Clear and Green,36760,2,5497.000000,112.500000,266.0
BTR,Primary,Non-Hub,PUBLIC,PUBLIC,1250.0,NO,Clear and Green,55331,3,6101.333333,125.000000,150.0


In [60]:
flights_fe = pd.merge(left=flights_fe,right=airport_details,right_index=True,left_on='origin')
flights_fe = pd.merge(left=flights_fe,right=airport_details,right_index=True,left_on='dest',suffixes=("_or","_des"))

In [61]:
flights_fe.shape

(129720, 60)

## Date, hour, week

In [62]:
# flights_modified['arr_year'] = flights_modified['arr_date_time'].dt.year
flights_fe['arr_month'] = flights_fe['arr_date_time'].dt.month
flights_fe['arr_day_of_week'] = flights_fe['arr_date_time'].dt.day_of_week
flights_fe['arr_hour'] = flights_fe['arr_date_time'].dt.hour


# flights_modified['dep_year'] = flights_modified['dep_date_time'].dt.year
flights_fe['dep_month'] = flights_fe['dep_date_time'].dt.month
flights_fe['dep_day_of_week'] = flights_fe['dep_date_time'].dt.day_of_week
flights_fe['dep_hour'] = flights_fe['dep_date_time'].dt.hour

In [63]:
flights_fe.shape

(129720, 66)

## Carrier Profits in 2019

## Carrier employment satisfaction

## carrier cargo tie-ups

## Save the file

In [65]:
flights_fe.to_csv(r"../data/interim/2_feature_engineered/test_data.csv",index=False)